# Amazon - User Playlist Engagement for Discovery

```SQL
CREATE TABLE playlists (
    playlist_id INTEGER,
    playlist_name VARCHAR,
    number_of_tracks INTEGER
);

DROP TABLE playlist_engagement;
CREATE TABLE playlist_engagement (
    playlist_id INTEGER,
    user_id INTEGER,
    listening_time_minutes FLOAT,
    engagement_date DATE
);

INSERT INTO playlists (playlist_id, playlist_name, number_of_tracks)
VALUES
    (1, 'Chill Vibes', 15),
    (2, 'Workout Hits', 10),
    (3, 'Top 50', 50),
    (4, 'Indie Discoveries', 8),
    (5, 'Jazz Essentials', 20),
    (6, 'Classical Mornings', 12),
    (7, 'Party Anthems', 25),
    (8, 'Acoustic Sessions', 18),
    (9, 'Rock Classics', 30),
    (10, 'Pop Culture', 22);

INSERT INTO playlist_engagement (playlist_id, user_id, listening_time_minutes, engagement_date)
VALUES
    (1, 101, 45, '2024-10-05'),
    (1, 102, 30, '2024-10-06'),
    (2, 103, 25, '2024-10-07'),
    (2, 104, 20, '2024-10-08'),
    (3, 105, 120, '2024-10-09'),
    (3, 106, 110, '2024-10-10'),
    (4, 107, 15, '2024-10-11'),
    (4, 108, 10, '2024-10-12'),
    (5, 109, 60, '2024-10-13'),
    (5, 110, 55, '2024-10-14'),
    (6, 111, 40, '2024-10-15'),
    (6, 112, 35, '2024-10-16'),
    (7, 113, 75, '2024-10-17'),
    (7, 114, 70, '2024-10-18'),
    (8, 115, 50, '2024-10-19'),
    (8, 116, 45, '2024-10-20'),
    (9, 117, 95, '2024-10-21'),
    (9, 118, 90, '2024-10-22'),
    (10, 119, 65, '2024-10-23'),
    (10, 120, 60, '2024-10-24');
```

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_playlist = pd.read_csv('Data/008/playlists.csv')
df_engagement = pd.read_csv('Data/008/playlist_engagement.csv')

df_playlist.head()

,playlist_id,playlist_name,number_of_tracks
0,1,Chill Vibes,15
1,2,Workout Hits,10
2,3,Top 50,50
3,4,Indie Discoveries,8
4,5,Jazz Essentials,20


In [3]:
df_engagement.head()

,playlist_id,user_id,listening_time_minutes,engagement_date
0,1,101,45,2024-10-05
1,1,102,30,2024-10-06
2,2,103,25,2024-10-07
3,2,104,20,2024-10-08
4,3,105,120,2024-10-09


# Pregunta 1

### El equipo de recomendaciones de Amazon Music quiere saber qué listas de reproducción tienen la menor cantidad de canciones. ¿Puedes encontrar la lista (o listas) con el número mínimo de canciones?

In [6]:
min_tracks = df_playlist['number_of_tracks'].min()

resultado = df_playlist[df_playlist['number_of_tracks'] == min_tracks]

resultado

,playlist_id,playlist_name,number_of_tracks
3,4,Indie Discoveries,8


```SQL
SELECT
    playlist_name
FROM playlists
WHERE number_of_tracks = (SELECT MIN(number_of_tracks) FROM playlists);
```

# Pregunta 2

### Estamos interesados en entender el nivel de engagement de las listas de reproducción. Específicamente, queremos identificar qué lista tiene el promedio de tiempo de escucha más bajo por canción. Esto implica calcular el tiempo total de escucha para cada lista en octubre de 2024 y luego normalizarlo dividiéndolo por el número de canciones en esa lista. ¿Puedes proporcionar el nombre de la lista con el valor más bajo basado en este cálculo?

In [8]:
# 1. Sumamos el tiempo de escucha total por ID
total_listening = df_engagement.groupby('playlist_id')['listening_time_minutes'].sum()

# 2. Unimos con la tabla de playlists (que tiene los nombres y n_tracks únicos)
df_analysis = pd.merge(df_playlist, total_listening, on='playlist_id')

# 3. Calculamos la métrica
df_analysis['ratio'] = df_analysis['listening_time_minutes'] / df_analysis['number_of_tracks']

# 4. Obtenemos el nombre de la menor
respuesta = df_analysis.sort_values('ratio').iloc[0]['playlist_name']

respuesta

'Indie Discoveries'

```SQL
SELECT
    p.playlist_name,
    SUM(e.listening_time_minutes)/p.number_of_tracks AS time_per_track
FROM playlists p
JOIN playlist_engagement e ON p.playlist_id = e.playlist_id
WHERE e.engagement_date BETWEEN '2024-10-01' AND '2024-10-31'
GROUP BY p.playlist_id, p.playlist_name, p.number_of_tracks
ORDER BY time_per_track ASC
LIMIT 1;
```

# Pregunta 3

### Para optimizar nuestras recomendaciones, necesitamos el tiempo de escucha promedio mensual por oyente para cada lista de reproducción en octubre de 2024. Para mejorar la legibilidad, por favor redondea hacia abajo el tiempo de escucha promedio al número entero más cercano.

In [12]:
# 1. Tiempo total por playlist
total_time = df_engagement.groupby('playlist_id')['listening_time_minutes'].sum()

# 2. Usuarios únicos por playlist
unique_users = df_engagement.groupby('playlist_id')['user_id'].nunique()

# 3. Métrica final con redondeo hacia abajo (usando astype)
avg_per_listener = (total_time / unique_users).astype(int)

# Para verlo bonito con nombres:
df_res = avg_per_listener.reset_index(name='avg_listening_time')
df_final = pd.merge(df_res, df_playlist[['playlist_id', 'playlist_name']], on='playlist_id')

df_final

,playlist_id,avg_listening_time,playlist_name
0,1,37,Chill Vibes
1,2,22,Workout Hits
2,3,115,Top 50
3,4,12,Indie Discoveries
4,5,57,Jazz Essentials
5,6,37,Classical Mornings
6,7,72,Party Anthems
7,8,47,Acoustic Sessions
8,9,92,Rock Classics
9,10,62,Pop Culture


```SQL
SELECT
    p.playlist_name,
    FLOOR(SUM(e.listening_time_minutes) / COUNT(DISTINCT e.user_id)) AS avg_listening_time
FROM playlists p
JOIN playlist_engagement e ON p.playlist_id = e.playlist_id
WHERE e.engagement_date >= '2024-10-01' AND e.engagement_date <= '2024-10-31'
GROUP BY p.playlist_id, p.playlist_name;
```